# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [21]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nadi,-17.8000,177.4167,301.15,69,40,4.63,FJ,1686619172
1,1,lionel town,17.8101,-77.2406,302.09,82,6,4.25,JM,1686619172
2,2,taoudenni,22.6783,-3.9836,307.60,10,51,3.77,ML,1686619172
3,3,sikasso,10.9167,-7.0000,297.93,80,13,1.97,ML,1686619172
4,4,treinta y tres,-33.2333,-54.3833,277.64,81,7,2.69,UY,1686619172


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    size='Humidity',
    c='Humidity',
    colormap='Blues',
    colorbar=True,
    alpha=0.7,
    hover_cols=['City', 'Humidity'],
    xlabel='Longitude',
    ylabel='Latitude',
    title='City Map - Humidity'
)
# Display the map
map_plot

:Scatter   [Lng]   (Lat,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[
    (city_data_df['Max Temp'] < 350) &
    (city_data_df['Wind Speed'] > 2) &
    (city_data_df['Humidity'] < 80)
]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

#Display sample data
filtered_cities_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nadi,-17.8000,177.4167,301.15,69,40,4.63,FJ,1686619172
2,2,taoudenni,22.6783,-3.9836,307.60,10,51,3.77,ML,1686619172
7,7,adamstown,-25.0660,-130.1015,293.44,64,27,3.44,PN,1686618745
10,10,kittila,67.6647,24.8936,286.23,54,0,4.12,FI,1686619174
11,11,margaret river,-33.9500,115.0667,288.57,71,100,6.83,AU,1686618767


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,nadi,FJ,-17.8000,177.4167,69,
2,taoudenni,ML,22.6783,-3.9836,10,
7,adamstown,PN,-25.0660,-130.1015,64,
10,kittila,FI,67.6647,24.8936,54,
11,margaret river,AU,-33.9500,115.0667,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "filter": "category eq 'hotel'",
    "bias": "proximity",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] += f",circle({longitude},{latitude},{radius})"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    data = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
nadi - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
kittila - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
sitka - nearest hotel: No hotel found
wladyslawowo - nearest hotel: No hotel found
labytnangi - nearest hotel: No hotel found
luuq - nearest hotel: No hotel found
mumford - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
whitehorse - nearest hotel: No hotel found
hadibu - nearest hotel: No hotel found
vadso - nearest hotel: No hotel found
zouerate - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
ilulissat - nearest hotel: No hotel found
muynoq - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
iqaluit - nearest hotel: No hotel found
mount isa - nearest hotel: No hotel found
yelizovo - nearest hotel: No hotel found
micheweni - nearest hotel: N

,City,Country,Lat,Lng,Humidity,Hotel Name
0,nadi,FJ,-17.8000,177.4167,69,No hotel found
2,taoudenni,ML,22.6783,-3.9836,10,No hotel found
7,adamstown,PN,-25.0660,-130.1015,64,No hotel found
10,kittila,FI,67.6647,24.8936,54,No hotel found
11,margaret river,AU,-33.9500,115.0667,71,No hotel found
...,...,...,...,...,...,...
564,goth radhan,PK,27.1833,67.9500,44,No hotel found
565,kodinsk,RU,58.6881,99.1844,48,No hotel found
567,slave lake,CA,55.2834,-114.7690,41,No hotel found
568,khash,IR,28.2211,61.2158,19,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

map_plot = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Humidity",
    size="Humidity",
    hover_cols=["City", "Country", "Hotel Name"],
    alpha=0.7,
    cmap="viridis",
    title="Hotel Map",
    xlabel="Longitude",
    ylabel="Latitude",
)

# Display the map
map_plot


:Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)